In [1]:
using Pkg
Pkg.add(url = "https://github.com/MolloiLab/imageToolBox.jl")


    Updating git-repo `https://github.com/MolloiLab/imageToolBox.jl`


    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


Precompiling 

...

  ✓ PyCall


  ✓ Pluto


  ✓ Plots


  ✓ Plots → FileIOExt


  ✓ Plots → UnitfulExt
  52 dependencies successfully precompiled in 58 seconds. 341 already precompiled.
  1 dependency had warnings during precompilation:
┌ Pluto [c3e4b0f8-55cb-11ea-2926-15256bba5781]
│  ┌ Info: 
│  │ 
│  │     Welcome to Pluto v0.19.27 🎈
│  │     Start a notebook server using:
│  │ 
│  │   julia> Pluto.run()
│  │ 
│  │     Have a look at the FAQ:
│  │     https://github.com/fonsp/Pluto.jl/wiki
│  └ 
└  


In [2]:
using imageToolBox
using Plots

In [3]:
label_dir= "/media/molloi-lab/2TB/Clean_Dataset_full"
res_dir = "/media/molloi-lab/1TB/Output"

"/media/molloi-lab/1TB/Output"

In [4]:
#Run time: 5min
SIDs = readdir(label_dir)
s = size(SIDs)[1]
data = Array{Any, 2}(undef, s, 2)
@Threads.threads for i = 1:s
    sid = SIDs[i]
    dir1 = joinpath(label_dir, sid)
    dir2 = joinpath(res_dir,sid)
    ct_gt, ct_pred = 0,0
    for f in readdir(dir1)
        f1_name, f1_ext = splitext(f)
        if f1_ext == ".png"
            # push!(())
            path1 = joinpath(dir1, f)
            path2 = joinpath(dir2, f1_name*".pred.png")
            if isfile(path2)
                ct_gt += sum(Int.(read_png(path1)))
                ct_pred += sum(Int.(read_png(path2)))
            end
        end
    end
    # record
    data[i, 1] = ct_gt
    data[i, 2] = ct_pred
end

In [5]:
keepidx = []
outputsid= []
drop_idxes = []
t = 3
for i in 1:s
    # if abs(data[i, 2] - data[i, 1]) < 3000 || (data[i,1] == 0 && data[i, 2] < 20000) || t > data[i,1] / data[i, 2] > 1/t 
    if abs(data[i, 2] - data[i, 1]) <= 75000
        push!(keepidx, i)
    else
        push!(drop_idxes, i)
        push!(outputsid, SIDs[i])
    end
end
@info "$(size(drop_idxes)[1]) dropped."

┌ Info: 102 dropped.
└ @ Main /home/molloi-lab/Desktop/Project BAC/BAC project/0.1_visualize.ipynb:14


In [6]:
scatter(data[keepidx, 1], data[keepidx, 2], xlabel="Readers' roi", ylabel="Prediction", title="BAC area")
plot!(data[keepidx, 1], data[keepidx, 1], label="y = x", linestyle=:dash)
savefig("BAC_area_plot_all.png")

"/home/molloi-lab/Desktop/Project BAC/BAC project/BAC_area_plot_all.png"

In [7]:
data_out = Array{Any, 2}(undef, size(drop_idxes)[1], 3)
data_out[:, 1] = SIDs[drop_idxes]
data_out[:, 2] = data[drop_idxes, 1]
data_out[:, 3] = data[drop_idxes, 2]
save_to_csv(["SID", "area_reader", "area_pred"], data_out, "outliers.csv")

"outliers.csv"